In [3]:
from numpy import *

In [4]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

In [5]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

In [7]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!") % word
    return returnVec

In [8]:
listOPosts, listClasses = loadDataSet()

In [9]:
myVocabList = createVocabList(listOPosts)

In [10]:
myVocabList

['please',
 'dalmation',
 'so',
 'take',
 'flea',
 'buying',
 'maybe',
 'steak',
 'garbage',
 'ate',
 'mr',
 'park',
 'love',
 'licks',
 'to',
 'food',
 'cute',
 'help',
 'has',
 'my',
 'dog',
 'stupid',
 'him',
 'quit',
 'problems',
 'worthless',
 'stop',
 'I',
 'is',
 'how',
 'posting',
 'not']